Use Kaggle sample:
https://www.kaggle.com/kakauandme/tensorflow-deep-nn

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Deal with Data

In [2]:
# Import Data
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

/Users/lin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.train.images.shape # 55000, 784
mnist.train.labels.shape # 55000, 10
mnist.validation.images.shape # 5000, 784
mnist.validation.labels.shape # 5000, 10
mnist.test.images.shape # 10000, 784
mnist.test.labels.shape  # 10000, 10

(10000, 10)

In [4]:
image_size = mnist.train.images.shape[1] # 784
labels_count = mnist.train.labels.shape[1] # 10
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)

# Construct Neural Nework Structure
---
Neural Network Process: <br/>
image -> convolution -> max pooling -> convolution -> max pooling -> fully connected -> fully connected -> classifier

---
Ref:
* https://brohrer.mcknote.com/zh-Hant/how_machine_learning_works/how_convolutional_neural_networks_work.html
* http://cs231n.github.io/
* https://www.youtube.com/watch?v=tjcgL5RIdTM&list=PLXO45tsB95cKI5AIlf5TxxFPzb-0zeVZ8&index=26

* Things should notice for Weight initialization:

Note that we do not know what the final value of every weight should be in the trained network, but with proper data normalization it is reasonable to assume that approximately half of the weights will be positive and half of them will be negative. 
A reasonable-sounding idea then might be to set all the initial weights to zero, which we expect to be the “best guess” in expectation. This turns out to be a mistake, because if every neuron in the network computes the same output, then they will also all compute the same gradients during backpropagation and undergo the exact same parameter updates. In other words, there is no source of asymmetry between neurons if their weights are initialized to be the same.
Therefore, **we still want the weights to be very close to zero, but as we have argued above, not identically zero. As a solution, it is common to initialize the weights of the neurons to small numbers and refer to doing so as symmetry breaking.**

* what is weight: http://cs231n.github.io/neural-networks-1/#nn
* concept of weight and bias: y=ax+b (a: weight; b: bias)

In [5]:
# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # create normal distribution
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# convolution: https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-04-CNN2/
def conv2d(x, W):
    # x: all inputs; W: weight
    # strides[1,x_movement,y_movement,1]: the output is the same size as the input
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') 

# pooling : extract the maximum value
# [[0,3],[4,2]] => 4

def max_pool_2x2(x):
    # strides=[1,2,2,1] -> make 2(1+1_xmovement)*2(1+1_ymovement) into 1*1 -> squize the image 
    # ksize: extract info from 2*2 
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
if __name__ == '__main__':
    x = tf.placeholder(tf.float32, [None,784])
    W_conv1 = weight_variable([5, 5, 1, 32]) # patch:5*5; in size:1; out size:32
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1,image_width,image_height,1]) # -1: n_sample 1: our display image is in black/white 
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)   # outpt size of h_convl will be 28*28*32
    h_pool1 = max_pool_2x2(h_conv1)     # output size of h_pool will be 14*14*32

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # outpt size of h_convl will be 14*14*64
    h_pool2 = max_pool_2x2(h_conv2) # output size of h_pool will be 7*7*64
    
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) # [n_sample, 7,7,64] -> [n_sample, 7*7*64]
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # Output layer
    W_fc2 = weight_variable([1024, labels_count]) #labels_count: contains 0-9
    b_fc2 = bias_variable([10])
    y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    # Evaluation
    # cross_entropy: https://www.tensorflow.org/get_started/mnist/beginners
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    init = tf.initialize_all_variables()
    
    sess = tf.Session()
    sess.run(init)
    
    for i in range(20000):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if i%100 == 0:
            
            train_accuracy = accuracy.eval(session=sess,feed_dict={x:batch_xs, y_: batch_ys, keep_prob: 1.0})
            print("step %d, training accuracy %.3f"%(i, train_accuracy))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print("test accuracy %g"%accuracy.eval(session=sess,feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.160
step 100, training accuracy 0.880
step 200, training accuracy 0.920
step 300, training accuracy 0.860
step 400, training accuracy 0.880
step 500, training accuracy 0.940
step 600, training accuracy 0.960
step 700, training accuracy 0.960
step 800, training accuracy 0.960
step 900, training accuracy 0.960
step 1000, training accuracy 0.940
step 1100, training accuracy 1.000
step 1200, training accuracy 0.940
step 1300, training accuracy 0.940
step 1400, training accuracy 0.980
step 1500, training accuracy 0.960
step 1600, training accuracy 1.000
step 1700, training accuracy 0.960
step 1800, training accuracy 0.980
step 1900, training accuracy 0.960
step 2000, training accuracy 0.980
step 2100, training accuracy 0.980
step 2200, training accuracy 0.980
step 2300, training accuracy 1.000
step 2400, training accuracy 1.000
step 2500, training accuracy 0.920
step 2600, training accuracy

# Pipeline (include different model) 9:06 starts -> 10:21

In [11]:
# install: http://docs.seldon.io/python-package.html
from seldon.tensorflow_wrapper import TensorFlowWrapper
from sklearn.pipeline import Pipeline
from seldon.pipeline.util import Pipeline_wrapper
tfw = TensorFlowWrapper(sess,tf_input=x,tf_output=y_conv,tf_constants=[(keep_prob,1.0)],target="y",target_readable="class",excluded=['class'])
p = Pipeline([('deep_classifier',tfw)])
pw = Pipeline_wrapper()
pw.save_pipeline(p,'deep_mnist_pipeline')

ModuleNotFoundError: No module named 'seldon'